In [1]:
import torch 
from src.evaluate import evaluate
from src.utils import dboxes300_coco, Encoder

In [2]:
torch.cuda.empty_cache()
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)

Using cache found in /home/bohumil/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [2]:
import argparse
from src.data import get_val_dataset, get_val_dataloader, get_coco_ground_truth

COCO_PATH = "/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/"

parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])
args.data = COCO_PATH

dataset = get_val_dataset(args)
len(dataset)

4952

In [3]:
type([500])

list

In [4]:
split1, _ = torch.utils.data.random_split(dataset, [50, 4952-50])

In [5]:
args.eval_batch_size = 10
args.num_workers = 1
args.distributed = False

dataloader = get_val_dataloader(split1, args)
truth = get_coco_ground_truth(args)

loading annotations into memory...
Done (t=0.78s)
creating index...
index created!


## Base precision

In [6]:
from src.evaluate import evaluate
from src.utils import dboxes300_coco, Encoder

dboxes = dboxes300_coco()
encoder = Encoder(dboxes)
inv_map = {v: k for k, v in dataset.label_map.items()}

args.no_cuda = True
args.amp = False
args.local_rank = 0

In [6]:
from src.evaluate import evaluate
from src.utils import dboxes300_coco, Encoder

dboxes = dboxes300_coco()
encoder = Encoder(dboxes)
inv_map = {v: k for k, v in dataset.label_map.items()}

args.no_cuda = True
args.amp = False
args.local_rank = 0

evaluate(ssd_model, dataloader, truth, encoder, inv_map, args)

Parsing batch: 24/25
Predicting Ended, total time: 134.34 s
Loading and preparing results...
Converting ndarray to lists...
(36524, 7)
0/36524
DONE (t=0.27s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=12.11s).
Accumulating evaluation results...
DONE (t=1.82s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.030
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.050
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.014
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.052
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.024
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.036
 Average Recall     (AR) @[ IoU=0.50:

0.030495475224678346

## Wrapped version for quantizer

In [7]:
torch.save(ssd_model.state_dict(), './SSD_orig_state.st')

In [7]:
from src.model import SSD300 , ResNet

ssd_dist = SSD300()
ssd_dist.load_state_dict(torch.load('./SSD_orig_state.st'))

<All keys matched successfully>

Getting stats for quantizer

In [8]:
torch.cuda.is_available()

True

In [9]:
import os
import torch
from distiller.data_loggers import collect_quant_stats, QuantCalibrationStatsCollector
import distiller

#man_model = torch.load('./manual.checkpoint.pth.tar')
torch.cuda.empty_cache()
distiller.utils.assign_layer_fq_names(ssd_dist)
collector = QuantCalibrationStatsCollector(ssd_dist)

stats_file = './acts_quantization_stats.yaml'

if not os.path.isfile(stats_file):
    def eval_for_stats(model):
        evaluate(model, dataloader, truth, encoder, inv_map, args)
    collect_quant_stats(ssd_dist, eval_for_stats, save_dir='.')


In [10]:
from copy import deepcopy
from distiller.quantization import PostTrainLinearQuantizer

def make_quantizer(cpu_model, config_file_path):
    """returns stats_before_prepare, quantizer"""
    parser = argparse.ArgumentParser()
    distiller.quantization.add_post_train_quant_args(parser)
    args = parser.parse_args(args=[])
    
    #this needs to be defined in config file
    # args.qe_stats_file = os.path.expanduser(stats_file)
    
    #config present, ALL OTHER args are IGNORED
    args.qe_config_file = os.path.expanduser(config_file_path)
    
    cp = deepcopy(cpu_model)
    quantizer = PostTrainLinearQuantizer.from_args(cp, args)
    
    # Quantizer magic
    stats_before_prepare = deepcopy(quantizer.model_activation_stats)
    
    # dummy input of (batch_size, height, width, depth)
    # https://towardsdatascience.com/understanding-input-and-output-shapes-in-convolution-network-keras-f143923d56ca
    dummy_input = (torch.zeros(1,3,300,300) )

    quantizer.prepare_model(dummy_input)
    
    return stats_before_prepare, quantizer

In [16]:
stats_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/acts_quantization_stats.yaml'
config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/ssd_coco_post_train.yaml'
stats_before_quantization8, quantizer8 = make_quantizer(ssd_dist,config_file)

config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/ssd_coco_post_train_4bit.yaml'
stats_before_quantization4, quantizer4 = make_quantizer(ssd_dist,config_file)

config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/ssd_coco_post_train_2bit.yaml'
stats_before_quantization2, quantizer2 = make_quantizer(ssd_dist,config_file)

quantizer8.model

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.96 GiB total capacity; 1016.89 MiB already allocated; 1.75 MiB free; 109.11 MiB cached)

### 8 bit kvantizacia

```python
bits_activations: 8
bits_parameters: 8
bits_accum: 16
```

In [11]:
torch.cuda.empty_cache()
stats_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/acts_quantization_stats.yaml'
config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/ssd_coco_post_train.yaml'
# ssd_dist = ssd_dist.cpu()
stats_before_quantization8, quantizer8 = make_quantizer(ssd_dist,config_file)

In [12]:
ssd_dist

SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): DistillerBottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-0

In [13]:
quantizer8.model

SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): RangeLinearQuantParamLayerWrapper(
        weights_quant_settings=(num_bits=8 ; quant_mode=ASYMMETRIC_UNSIGNED ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=True)
        output_quant_settings=(num_bits=8 ; quant_mode=ASYMMETRIC_UNSIGNED ; clip_mode=AVG ; clip_n_stds=None ; clip_half_range=True ; per_channel=False)
        accum_quant_settings=(num_bits=16 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
          inputs_quant_auto_fallback=True, forced_quant_settings_for_inputs=None
        scale_approx_mult_bits=None
        preset_activation_stats=True
          output_scale=62.817833, output_zero_point=0.000000
        weights_scale=PerCh, weights_zero_point=PerCh
        (wrapped_module): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      )
      (1): Identity()
      (2): Identity()
      (3):

In [15]:
# model = deepcopy(quantizer8.model).to('cpu')
# model = model.cpu()
args.no_cuda = False

evaluate(quantizer8.model, dataloader, truth, encoder, inv_map, args)

Parsing batch: 4/5
Predicting Ended, total time: 38.41 s
Loading and preparing results...
Converting ndarray to lists...
(4662, 7)
0/4662
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.71s).
Accumulating evaluation results...
DONE (t=1.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 |

3.8690418212645517e-07

### 4 bit kvantizacia
```python
bits_activations: 4
bits_parameters: 4
bits_accum: 8
```

In [15]:
config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/ssd_coco_post_train_4bit.yaml'
stats_before_quantization4, quantizer4 = make_quantizer(ssd_dist,config_file)

evaluate(quantizer4.model, dataloader, truth, encoder, inv_map, args)

KeyboardInterrupt: 

### 2 bit kvantizacia
```python
bits_activations: 2
bits_parameters: 2
bits_accum: 2
```

In [ ]:
config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/ssd_coco_post_train_2bit.yaml'
stats_before_quantization2, quantizer2 = make_quantizer(ssd_dist,config_file)
evaluate(quantizer2.model, dataloader, truth, encoder, inv_map, args)